In [1]:
!tesseract --list-langs

List of available languages (163):
afr
amh
ara
asm
aze
aze_cyrl
bel
ben
bod
bos
bre
bul
cat
ceb
ces
chi_sim
chi_sim_vert
chi_tra
chi_tra_vert
chr
cos
cym
dan
deu
div
dzo
ell
eng
enm
epo
equ
est
eus
fao
fas
fil
fin
fra
frk
frm
fry
gla
gle
glg
grc
guj
hat
heb
hin
hrv
hun
hye
iku
ind
isl
ita
ita_old
jav
jpn
jpn_vert
kan
kat
kat_old
kaz
khm
kir
kmr
kor
kor_vert
lao
lat
lav
lit
ltz
mal
mar
mkd
mlt
mon
mri
msa
mya
nep
nld
nor
oci
ori
osd
pan
pol
por
pus
que
ron
rus
san
script/Arabic
script/Armenian
script/Bengali
script/Canadian_Aboriginal
script/Cherokee
script/Cyrillic
script/Devanagari
script/Ethiopic
script/Fraktur
script/Georgian
script/Greek
script/Gujarati
script/Gurmukhi
script/HanS
script/HanS_vert
script/HanT
script/HanT_vert
script/Hangul
script/Hangul_vert
script/Hebrew
script/Japanese
script/Japanese_vert
script/Kannada
script/Khmer
script/Lao
script/Latin
script/Malayalam
script/Myanmar
script/Oriya
script/Sinhala
script/Syriac
script/Tamil
script/Telugu
script/Thaana
script/Th

In [2]:
!pip install pytesseract

In [3]:
!pip install pdf2image

In [4]:
pip install natsort

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import libraries
from PIL import Image
import pytesseract
import os ## navitage, create directories
import shutil ## to delete the image folders with their imgs
from pdf2image import convert_from_path ## to turn pdf to image
import glob ## to glob files into a list
from pathlib import Path ## to specify path to your files
from natsort import natsorted, ns ## natural sorting
import os      ##to sort files
import re

# testing conversions on one pdf

In [6]:
##path for a test pdf
PDF_file1 = "found_docs/Jun-20.pdf"

In [7]:
pages = convert_from_path(PDF_file1, 300) ## 500 dpi. balance between hi-res and computation power
print(type(pages))
pages

<class 'list'>


[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2550x3301 at 0x7F98AFBA2280>]

In [8]:
## Take your PDF and convert each page to a JPEG

# Store all the pages of the PDF in a variable
pages = convert_from_path(PDF_file1, 400) ## 500 dpi. balance between hi-res and computation power
print(pages)
print(f"Give me a second...converting '{PDF_file1}' to a JPEG")

# Counter to store images of each page of PDF to image
image_counter = 1

# Iterate through all the pages stored above
for page in pages:
    ## Declaring filename for each page of PDF as JPG
    ## remove the .pdf extention (last 4 characters)
    filename = str(PDF_file1[ : -4])+"_page_"+str(image_counter)+".jpg"
    print(f"Here we go: {filename}")
    print(type(filename))
    # Save the image of the page in system
    page.save(filename, 'JPEG') ## here we tell it to be a "jpeg" file
    # Increment the counter to update filename
    image_counter+=1

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=3400x4400 at 0x7F98B4606FA0>]
Give me a second...converting 'found_docs/Jun-20.pdf' to a JPEG
Here we go: found_docs/Jun-20_page_1.jpg
<class 'str'>


In [9]:
##putting in a path for the test document
img_path = "found_docs/Jun-20*.jpg"
myimg = sorted(glob.glob(img_path))
myimg

['found_docs/Jun-20_page_1.jpg']

In [10]:
outfile = "Juun-20.txt"

In [11]:
## Open each Jpeg, OCR and append text to output file

with open(outfile, "a") as my_conversion_file:    ##using a to append, not w because w will write over it
  ## grab all jpegs
  img_files = sorted(glob.glob(img_path))
  print(img_files)
  counter = 0
    ## name each file
  for img_file in img_files:
    counter+=1
    print(f"Converting image {counter} of {len(img_files)} to text")
    text = f"FILE_Info: {img_file} \n\n\n + {str(pytesseract.image_to_string(Image.open(img_file)))}" 

  ## In many PDFs, at line ending, if a word can't
    ## be written fully, a 'hyphen' is added.
    ## To remove this, we replace every '-\n' to ''.
    text = text.replace('-\n', '')

    my_conversion_file.write(text)

print(f"Your converted PDFs are waiting in {outfile}!")

['found_docs/Jun-20_page_1.jpg']
Converting image 1 of 1 to text
Your converted PDFs are waiting in Juun-20.txt!


# functions to use on multiple pdfs

In [12]:
### FUNCTION to convert pdf to jpg
def to_jpg(PDF_file, resolution):
    '''
    arguments
    - PDF_file = name of pdf file to OCR
    - resolution = resolution of conversion of pdf to jpeg
    
    '''
    ## store in a folder for conversion
    conversion_path = Path('conversion_img/')
    conversion_path.mkdir(exist_ok=True)

    '''
    Part #1 : Converting PDF to images
    '''

    # Store all the pages of the PDF in a variable
    pages = convert_from_path(PDF_file, resolution)
    print(f"Give me a second...converting '{PDF_file}' to a JPEG")

    # Counter to store images of each page of PDF to image
    image_counter = 1
    

    # Iterate through Sall the pages stored above
    for page in pages:
        ## Declaring filename for each page of PDF as JPG
        ## remove the .pdf extention (last 4 characters)
        ## remove the
        if re.search(folder_pattern, PDF_file) != None:
            PDF_file = re.split(folder_pattern, PDF_file)[1]
   
        filename = str(PDF_file[ : -4])+"_page_"+str(image_counter)+".jpg"
        print(f"{filename} has been converted to an image. Will OCR it now...")
        filename = os.path.join(conversion_path,filename)
        print(filename)
        # Save the image of the page in system
        page.save(filename, 'JPEG')

        # Increment the counter to update filename
        image_counter+=1

In [13]:
### FUNCTION TO OCR JPG

def jpg_to_txt(outfile, language = "eng"):

    '''
    Function that Recognizing text from the images using OCR
    Argument(s):
    - outfile = file name (as a string) to store your OCRed text 
    - language = tessaract language code. eng by default
    '''
    ## store in an output folder
    output_path = Path('output/')
    output_path.mkdir(exist_ok=True)
    conversion_path = Path('conversion_img/')

    # # # # Creating a text file to write the output
    outfile = os.path.join(output_path,outfile)
    # # #
    # # # # Open output file and append all info

    with open(outfile, "a") as f:
        ## grab all jpegs
        img_files = sorted(Path(conversion_path).glob('*.jpg'), reverse=False)

        ## path objects turn urls into strings and you can't sort numbers with underscores, etc
        ## the function below uses regex to find the numbers and turn them into integer.
        ## natsort library
        ## https://natsort.readthedocs.io/en/master/howitworks.html
        from natsort import natsorted, ns
        
        ## .as_posix() returns a string representation of the path with forward slashes (/)
        ## more info: https://docs.python.org/3/library/pathlib.html#pathlib.PurePath.as_posix
        ## if you receive a posix error, comment out next line and uncomment the one after
        img_files = [img_file for img_file in img_files] 
#         img_files = [img_file.as_posix() for img_file in img_files] 
#         print(f"img_files: {img_files}")
        img_files = natsorted(img_files, alg=ns.PATH)
#         print(img_files)

        counter = 0
        for img_file in img_files:
            counter+=1
            print(f"Converting image {counter} of {len(img_files)} to text!")
            text = f"\n\n\nFILE_Info: {img_file} \n\n\n \
             {str(pytesseract.image_to_string(Image.open(img_file), lang = language))}"
            
            ## In many PDFs, at line ending, if a word can't
            ## be written fully, a 'hyphen' is added.
            ## To remove this, we replace every '-\n' to ''.
            text = text.replace('-\n', '')
            # print(f"Here's the text: \n {text}")

            f.write(text)

            ## Delete the folder
        shutil.rmtree(conversion_path)
        return img_files
        

In [14]:
##function that calls on both functions above together
def ocr_file(PDF_file, outfile, pdf_path = "", resolution = 400 ):
    '''
    args:
    - pdf file name(as strings)
    - name of outfile (as strings)
    - path to pdf as string (default "" same folder)
    - resolution as int between 300 and 600
    '''
    to_jpg(PDF_file, resolution)
    jpg_to_txt(outfile)
    

In [15]:
##putting/sorting all documents from a folder to a list
path = "found_docs/*.pdf"
myfiles = sorted(glob.glob(path))
myfiles

['found_docs/Apr-19.pdf',
 'found_docs/Aug-19_Finance_Bulletin.pdf',
 'found_docs/Aug-20.pdf',
 'found_docs/Dec-19 Finance_Bulletin.pdf',
 'found_docs/Dec-20.pdf',
 'found_docs/FB_April_2020.pdf',
 'found_docs/Feb-19.pdf',
 'found_docs/Feb_2020_FB.pdf',
 'found_docs/Jul-19.pdf',
 'found_docs/Jul-20.pdf',
 'found_docs/Jun-19.pdf',
 'found_docs/Jun-20.pdf',
 'found_docs/Mar-19.pdf',
 'found_docs/Mar_2020_FB.pdf',
 'found_docs/Nov-19_Finance_Bulletin.pdf',
 'found_docs/Nov-20.pdf',
 'found_docs/Oct-19_Finance_Bulletin.pdf',
 'found_docs/Oct-20.pdf',
 'found_docs/Sep-20.pdf',
 'found_docs/Sept-19_Finance_Bulletin.pdf']

In [16]:
##An attempt to iterate through each item in the list file
for each in myfiles:
    ocr_file(each, "{each}.txt", "", 400)

Give me a second...converting 'found_docs/Apr-19.pdf' to a JPEG


NameError: name 'folder_pattern' is not defined

In [17]:
folder_pattern = re.compile(r".+\/")

In [18]:
##function that iterates through the list of files and then calls on the ocr_file function
def process_files(files_list, name_output_file, language = "eng", resolution = 400):
    '''
    This function iterates through a list that needs OCRing
    It taps all the functions above so we enter ALL the arguments here:
    Arguments:
    - files_list = name of list that must be iterated. You have globbed files into it.
    - outfile = file name (as a string) to store your OCRed text
    - resolution = resolution of scan as int between 300 and 600
    '''
#     checker(name_output_file)
    counter = 1
    len(files_list)
    for file in files_list:
        print(f"Grabbing '{file}' (file {counter} of {len(files_list)}) from your list....")
        counter += 1
        ocr_file(file, name_output_file, resolution)
    print(f"A text file called '{name_output_file}' is ready for you....")

In [19]:
##calls on the function to iterate through all the pdfs and puts them all in a single txt file
process_files(myfiles, "cal_revenue.txt")

Grabbing 'found_docs/Apr-19.pdf' (file 1 of 20) from your list....
Give me a second...converting 'found_docs/Apr-19.pdf' to a JPEG
Apr-19_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Apr-19_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'found_docs/Aug-19_Finance_Bulletin.pdf' (file 2 of 20) from your list....
Give me a second...converting 'found_docs/Aug-19_Finance_Bulletin.pdf' to a JPEG
Aug-19_Finance_Bulletin_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Aug-19_Finance_Bulletin_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'found_docs/Aug-20.pdf' (file 3 of 20) from your list....
Give me a second...converting 'found_docs/Aug-20.pdf' to a JPEG
Aug-20_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Aug-20_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'found_docs/Dec-19 Finance_Bulletin.pdf' (file 4 of 20) from your list....
Give me a second...converting 'found_docs/Dec-19 F

# Using REGEX to get the required text

In [20]:
## create patterns from regex expressions from regex101 that targets the required text
date_pat = re.compile ("^((JANUARY|FEBRUARY|MARCH|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER)\s\d{4})", re.I|re.M)
alcohol_pat = re.compile ("^(Alcoholic Beverages)\s(\w{2})\s(\d[\d]*[\W]?)", re.I|re.M)

In [21]:
##open and read the single and newly created txt file
with open("output/cal_revenue.txt", 'r') as tdoc:
        file = tdoc.read()

In [22]:
##puts the found patterns in a list
date_list = list(date_pat.findall(file))   
alcohol_list = list(alcohol_pat.findall(file))   
date_list

[('MARCH 2019', 'MARCH'),
 ('JULY 2019', 'JULY'),
 ('JULY 2020', 'JULY'),
 ('NOVEMBER 2019', 'NOVEMBER'),
 ('NOVEMBER 2020', 'NOVEMBER'),
 ('MARCH 2020', 'MARCH'),
 ('JANUARY 2019', 'JANUARY'),
 ('JANUARY 2020', 'JANUARY'),
 ('JUNE 2019', 'JUNE'),
 ('JUNE 2020', 'JUNE'),
 ('MAY\n2019', 'MAY'),
 ('MAY 2020', 'MAY'),
 ('FEBRUARY 2019', 'FEBRUARY'),
 ('FEBRUARY 2020', 'FEBRUARY'),
 ('OCTOBER 2019', 'OCTOBER'),
 ('OCTOBER 2020', 'OCTOBER'),
 ('SEPTEMBER 2019', 'SEPTEMBER'),
 ('SEPTEMBER 2020', 'SEPTEMBER'),
 ('AUGUST 2020', 'AUGUST'),
 ('AUGUST 2019', 'AUGUST')]

In [23]:
##the patterns comes out as a tuple
date_list
for a,b in date_list:
    print(a)

MARCH 2019
JULY 2019
JULY 2020
NOVEMBER 2019
NOVEMBER 2020
MARCH 2020
JANUARY 2019
JANUARY 2020
JUNE 2019
JUNE 2020
MAY
2019
MAY 2020
FEBRUARY 2019
FEBRUARY 2020
OCTOBER 2019
OCTOBER 2020
SEPTEMBER 2019
SEPTEMBER 2020
AUGUST 2020
AUGUST 2019


In [24]:
##unpack the tuples in the first list
date = []

for a,b in date_list:
    date.append(a)
date


['MARCH 2019',
 'JULY 2019',
 'JULY 2020',
 'NOVEMBER 2019',
 'NOVEMBER 2020',
 'MARCH 2020',
 'JANUARY 2019',
 'JANUARY 2020',
 'JUNE 2019',
 'JUNE 2020',
 'MAY\n2019',
 'MAY 2020',
 'FEBRUARY 2019',
 'FEBRUARY 2020',
 'OCTOBER 2019',
 'OCTOBER 2020',
 'SEPTEMBER 2019',
 'SEPTEMBER 2020',
 'AUGUST 2020',
 'AUGUST 2019']

In [25]:
##unpack the tuples in the second list
forecast = []
actual=[]
for a,b,c in alcohol_list:
    forecast.append(b)
    actual.append(c)
actual

['22 ',
 '35 ',
 '4]',
 '39 ',
 '35 ',
 '25 ',
 '38 ',
 '41 ',
 '28 ',
 '28 ',
 '24 ',
 '28 ',
 '22 ',
 '28 ',
 '33 ',
 '37 ',
 '29 ',
 '33 ',
 '34 ',
 '30 ']

In [26]:
import pandas as pd

In [27]:
## zip the variables and store in a list called txt_list
txt_list = []
for item in zip (date, forecast, actual):
    txt_list.append(item)
txt_list

[('MARCH 2019', 'QT', '22 '),
 ('JULY 2019', '38', '35 '),
 ('JULY 2020', '39', '4]'),
 ('NOVEMBER 2019', '33', '39 '),
 ('NOVEMBER 2020', '33', '35 '),
 ('MARCH 2020', '26', '25 '),
 ('JANUARY 2019', '41', '38 '),
 ('JANUARY 2020', '40', '41 '),
 ('JUNE 2019', '50', '28 '),
 ('JUNE 2020', '33', '28 '),
 ('MAY\n2019', '31', '24 '),
 ('MAY 2020', '31', '28 '),
 ('FEBRUARY 2019', '24', '22 '),
 ('FEBRUARY 2020', '25', '28 '),
 ('OCTOBER 2019', '33', '33 '),
 ('OCTOBER 2020', '33', '37 '),
 ('SEPTEMBER 2019', '33', '29 '),
 ('SEPTEMBER 2020', '33', '33 '),
 ('AUGUST 2020', '30', '34 '),
 ('AUGUST 2019', '30', '30 ')]

In [46]:
##sorts the list, alphabetically
txt_list_sort = natsorted(txt_list)
txt_list_sort

[('AUGUST 2019', '30', '30 '),
 ('AUGUST 2020', '30', '34 '),
 ('FEBRUARY 2019', '24', '22 '),
 ('FEBRUARY 2020', '25', '28 '),
 ('JANUARY 2019', '41', '38 '),
 ('JANUARY 2020', '40', '41 '),
 ('JULY 2019', '38', '35 '),
 ('JULY 2020', '39', '4]'),
 ('JUNE 2019', '50', '28 '),
 ('JUNE 2020', '33', '28 '),
 ('MARCH 2019', 'QT', '22 '),
 ('MARCH 2020', '26', '25 '),
 ('MAY\n2019', '31', '24 '),
 ('MAY 2020', '31', '28 '),
 ('NOVEMBER 2019', '33', '39 '),
 ('NOVEMBER 2020', '33', '35 '),
 ('OCTOBER 2019', '33', '33 '),
 ('OCTOBER 2020', '33', '37 '),
 ('SEPTEMBER 2019', '33', '29 '),
 ('SEPTEMBER 2020', '33', '33 ')]

In [47]:
##convert the list into a dataframe
df = pd.DataFrame (txt_list_sort)
df.columns = ["Date", "Alcohol_Forecast", "Alcohol_Actual"]
df


,Date,Alcohol_Forecast,Alcohol_Actual
0,AUGUST 2019,30,30
1,AUGUST 2020,30,34
2,FEBRUARY 2019,24,22
3,FEBRUARY 2020,25,28
4,JANUARY 2019,41,38
5,JANUARY 2020,40,41
6,JULY 2019,38,35
7,JULY 2020,39,4]
8,JUNE 2019,50,28
9,JUNE 2020,33,28


In [50]:
df.at[7,'Alcohol_Actual']= 41

In [48]:
df.at[10,'Alcohol_Forecast']= 27

In [51]:
df

,Date,Alcohol_Forecast,Alcohol_Actual
0,AUGUST 2019,30,30
1,AUGUST 2020,30,34
2,FEBRUARY 2019,24,22
3,FEBRUARY 2020,25,28
4,JANUARY 2019,41,38
5,JANUARY 2020,40,41
6,JULY 2019,38,35
7,JULY 2020,39,41
8,JUNE 2019,50,28
9,JUNE 2020,33,28


In [52]:
##convert the dateframe into a csv
file_name = "cal_alcohol_revenues.csv"
df.to_csv(file_name, encoding = "UTF-8", index = False)